# **Zero Shot Classification**

Install the relevant libraries required

In [1]:
!pip3 install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 211 kB 19.7 MB/s 
     |████████████████████████████████| 81 kB 5.2 MB/s 
     |████████████████████████████████| 93 kB 1.1 MB/s 
     |████████████████████████████████| 7.4 MB 23.2 MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13553 sha256=bde55697b0594166b3a17778985d62056e2356c1fb163ef8b0cce5c80bb57d11
  Stored in directory: /root/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3357 sha256=981b0d23679d841f2613fdab2bd5a76f82dd2739a49f517931fd44c051d2c68b
  Stored in directory: /root/.cache/pip/wheels/7f/d4/8f/6e2ca54744c9d7292d88ddb8d42876bcdab5e6d84a21c10346
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398404 sha256=5c0e0bd9ef21706d168c62bd525c2180b335aafc9e341ee6

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 31.6 MB/s 
     |████████████████████████████████| 163 kB 49.3 MB/s 
     |████████████████████████████████| 7.6 MB 38.3 MB/s 


In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


## Reading the annotated Pillars from Straits Times

In [97]:
import pandas as pd

df = pd.read_csv('data/ST_Pillars_Annotated.csv')
df = df.fillna('')
df.head()

,id,Title,Detail,Link,Update date,Publish date,Pillars,label
0,1363805,S'pore dedicates $25m to liver cancer research...,SINGAPORE - Singapore is dedicating $25 millio...,https://www.straitstimes.com/singapore/health/...,13/6/2022 16:59,13/6/2022 16:38,I,Innovate to Advance
1,1363825,S'pore dedicates $25m to liver cancer research...,SINGAPORE - Singapore is dedicating $25 millio...,https://www.straitstimes.com/singapore/health/...,13/6/2022 17:35,13/6/2022 17:35,I,Innovate to Advance
2,1368129,Non-surgical procedure for dialysis patients p...,SINGAPORE - Patients with end-stage kidney fai...,https://www.straitstimes.com/singapore/health/...,28/6/2022 19:00,28/6/2022 18:38,C,Care to Heal
3,1368131,Non-surgical procedure for dialysis patients p...,SINGAPORE - Patients with end-stage kidney fai...,https://www.straitstimes.com/singapore/health/...,28/6/2022 19:02,28/6/2022 19:02,C,Care to Heal
4,1357023,New programme to improve standard of emergency...,SINGAPORE - Every minute counts and a programm...,https://www.straitstimes.com/singapore/new-pro...,21/5/2022 13:01,21/5/2022 10:50,C,Care to Heal


In [98]:
from newspaper import Article

def news_reader(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()

    return article.text

## Obtaining the article full text

In [99]:
df['text'] = ''

for ind in df.index:
    df['text'][ind] = news_reader(df['Link'][ind])    


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [100]:
df_new = df[['text', 'label']]

In [101]:
df_new.head()

,text,label
0,SINGAPORE - Singapore is dedicating $25 millio...,Innovate to Advance
1,SINGAPORE - Singapore is dedicating $25 millio...,Innovate to Advance
2,SINGAPORE - Patients with end-stage kidney fai...,Care to Heal
3,SINGAPORE - Patients with end-stage kidney fai...,Care to Heal
4,SINGAPORE - Every minute counts and a programm...,Care to Heal


## Spliting the dataset to train and test

In [102]:
from sklearn.model_selection import train_test_split
dftrain, dftest = train_test_split(df_new, test_size=0.2)

In [44]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [84]:
from datasets import load_dataset, load_metric, Dataset
import numpy as np
import math
import datasets
from sklearn.metrics import classification_report

from transformers import pipeline
from tqdm.notebook import tqdm

In [85]:
train_dataset = Dataset.from_dict(dftrain)
test_dataset = Dataset.from_dict(dftest)
my_dataset_dict = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})

In [87]:
my_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 43
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 11
    })
})

In [89]:
print(my_dataset_dict["train"][0])

{'text': "SINGAPORE - Patients with end-stage kidney failure may soon no longer have to undergo surgery before beginning dialysis, thanks to a new procedure being piloted by Singapore General Hospital (SGH).\n\nAssociate Professor Tan Chieh Suai, head and senior consultant at SGH's department of renal medicine, announced this on Tuesday (June 28) and said that in 2020, 7,125 such patients - about three in four - chose to go for a form of dialysis known as haemodialysis, up from 4,270 in 2011.\n\nHaemodialysis involves using a dialysis machine and an artificial kidney to clean the blood of kidney patients.\n\nIn order for the blood to be extracted from patients, however, an access point needs to be created in their body.\n\nProf Tan said the best way of doing this is through the creation of what is known as an arteriovenous fistula (AVF), which involves joining a vein and an artery together and causing blood from the artery to flow through the vein.\n\nThis increases blood flow in the v

## Zero Shot Classification

In [4]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [106]:
df['Label'] = ''
df['Label_Score'] = ''

for ind in df.index:
    labels=["Care to Heal", "Educate to Empower", "Innovate to Advance"]
    out = classifier(df['text'][ind], labels, multi_label=True)

    df['Label'][ind] = out["labels"][0]
    df['Label_Score'][ind] = out["scores"][0]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [107]:
df.head()

,id,Title,Detail,Link,Update date,Publish date,Pillars,label,text,Label,Label_Score
0,1363805,S'pore dedicates $25m to liver cancer research...,SINGAPORE - Singapore is dedicating $25 millio...,https://www.straitstimes.com/singapore/health/...,13/6/2022 16:59,13/6/2022 16:38,I,Innovate to Advance,SINGAPORE - Singapore is dedicating $25 millio...,Care to Heal,0.463861
1,1363825,S'pore dedicates $25m to liver cancer research...,SINGAPORE - Singapore is dedicating $25 millio...,https://www.straitstimes.com/singapore/health/...,13/6/2022 17:35,13/6/2022 17:35,I,Innovate to Advance,SINGAPORE - Singapore is dedicating $25 millio...,Care to Heal,0.463861
2,1368129,Non-surgical procedure for dialysis patients p...,SINGAPORE - Patients with end-stage kidney fai...,https://www.straitstimes.com/singapore/health/...,28/6/2022 19:00,28/6/2022 18:38,C,Care to Heal,SINGAPORE - Patients with end-stage kidney fai...,Educate to Empower,0.566656
3,1368131,Non-surgical procedure for dialysis patients p...,SINGAPORE - Patients with end-stage kidney fai...,https://www.straitstimes.com/singapore/health/...,28/6/2022 19:02,28/6/2022 19:02,C,Care to Heal,SINGAPORE - Patients with end-stage kidney fai...,Educate to Empower,0.566656
4,1357023,New programme to improve standard of emergency...,SINGAPORE - Every minute counts and a programm...,https://www.straitstimes.com/singapore/new-pro...,21/5/2022 13:01,21/5/2022 10:50,C,Care to Heal,SINGAPORE - Every minute counts and a programm...,Care to Heal,0.638247


In [108]:
df.to_csv('data/ST_Pillars_Output.csv')

In [90]:
candidate_labels = ["Care to Heal", "Educate to Empower", "Innovate to Advance"]
predictions = []
for offset in tqdm(range(math.ceil(len(my_dataset_dict["test"])/16))):
    preds = classifier([my_dataset_dict["test"][16*offset+i]["text"] for i in range(16) if 16*offset+i<len(my_dataset_dict["test"])], candidate_labels)
    pred_labels = [pred["labels"][np.argmax(pred["scores"])] for pred in preds]
    predictions.extend(pred_labels)

  0%|          | 0/1 [00:00<?, ?it/s]

## Classification Report

In [91]:
print(classification_report([x["label"] for x in my_dataset_dict["test"]], predictions))

                     precision    recall  f1-score   support

       Care to Heal       0.50      0.20      0.29         5
 Educate to Empower       0.40      1.00      0.57         2
Innovate to Advance       0.25      0.25      0.25         4

           accuracy                           0.36        11
          macro avg       0.38      0.48      0.37        11
       weighted avg       0.39      0.36      0.32        11

